In [19]:
import pandas as pd
import os
import numpy as np
import cv2

In [16]:
import pandas as pd
import os

csv_path = "dataset/train.csv"
left_train_path = "dataset/train/left/"
right_train_path = "dataset/train/right/"
df_match = pd.read_csv(csv_path)

right_img_lst = []
# list all figures in the right folder
for filename in os.listdir(right_train_path):
    if os.path.isfile(os.path.join(right_train_path, filename)):
        right_img_lst.append(filename)

# extract image path and concatenate with image file path
anchor_img_lst = [filename + ".jpg" for filename in df_match['left']]
positive_img_lst = [filename + ".jpg" for filename in df_match['right']]

# create a list of tuples of image pairs
match_pairs = list(zip(anchor_img_lst, positive_img_lst))

In [17]:
# create a list of tuples of random unmatched image pairs
unmatch_pairs = []
for i in range(len(anchor_img_lst)):
    rand_idx_lst = []
    while len(rand_idx_lst) < 19:
        rand_idx = np.random.randint(0, len(right_img_lst))
        if rand_idx not in rand_idx_lst and right_img_lst[rand_idx] != positive_img_lst[i]:
            rand_idx_lst.append(rand_idx)
    unmatch_pairs.append([(anchor_img_lst[i], right_img_lst[index]) for index in rand_idx_lst])

# create triplet with one left anchor image, one right positive image, and one random negative image
triplet_pairs = []
for i in range(len(anchor_img_lst)):
    while True:
        rand_idx = np.random.randint(0, len(right_img_lst))
        if right_img_lst[rand_idx] != match_pairs[i][1]:
            triplet = [match_pairs[i][0], match_pairs[i][1], right_img_lst[rand_idx]]
            triplet_pairs.append(triplet)
            break

negative_img_lst = [filename[2] for filename in triplet_pairs]

In [20]:
# prepare image for the model
def prepare_image(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype("float32")
    img /= 255.0
    return img

# create a list of anchor, positive, and negative images
anchor_imgs = [prepare_image(os.path.join(left_train_path, filename)) for filename in anchor_img_lst]
positive_imgs = [prepare_image(os.path.join(right_train_path, filename)) for filename in positive_img_lst]
negative_imgs = [prepare_image(os.path.join(right_train_path, filename)) for filename in negative_img_lst]